In [1]:
import json
from helper import Verbose

In [2]:
Verbose(show_time=False)

In [3]:
json_file = open("../datasets/archived/ias-json.json")
json_data = json.load(json_file)

In [4]:
json_data[0].keys()

dict_keys(['Name', 'Service', 'Cadre', 'Allotment Year', 'Date of Birth', 'Date of Joining', 'Source of Recruitment', 'Gender', 'Place of Domicile', 'Mother Tongue', 'Languages Known', 'Retirement Reason', 'Qualification/University/Institute', 'Subject', 'Division', 'Designation/Level', 'Ministry/Department/Office/Location', 'Organisation', 'Experience(major/minor)', 'Period(From/To)', 'Dataset name', 'Clean Ids'])

In [5]:
def make_new_object(j_object):
    id_val = j_object["Clean Ids"]
    new_object = {}
    education_keys = ['Qualification/University/Institute', 'Subject', 'Division']
    experience_keys = ['Designation/Level', 'Ministry/Department/Office/Location', 'Organisation', 'Experience(major/minor)', 'Period(From/To)']
    for key in j_object.keys():
        if not (key in education_keys or key in experience_keys) and not key == "Clean Ids":
            new_object[key] = j_object[key]

    education_length = len(j_object["Subject"])
    edu_objects = []
    for i in range(education_length):
        edu_object = {}
        for key in education_keys:
            try:
                edu_object[key] = j_object[key][i]
            except:
                edu_object[key] = "N.A."
        
        edu_objects.append(edu_object)
    
    new_object["Education"] = edu_objects
    
    experience_length = len(j_object["Designation/Level"])
    exp_objects = []
    for i in range(experience_length):
        exp_object = {}
        for key in experience_keys:
            if key == "Experience(major/minor)":
                values = j_object[key][i].split("/")
                exp_object["Experience Major"] = values[0]
                try:
                    exp_object["Experience Minor"] = values[1]
                except:
                    exp_object["Experience Minor"] = "N.A."
            elif key == "Designation/Level":
                values = j_object[key][i].split("/")
                exp_object["Designation"] = values[0]
                try:
                    exp_object["Level"] = values[1]
                except:
                    exp_object["Level"] = "N.A."
            elif key == "Period(From/To)":
                values = j_object[key][i].split(" - ")
                exp_object["Period Start"] = values[0]
                try:
                    exp_object["Period End"] = values[1]
                except:
                    exp_object["Period End"] = "N.A."
            else:
                try:
                    exp_object[key] = j_object[key][i]
                except:
                    exp_object[key] = "N.A."
        
        exp_objects.append(exp_object)

    new_object["Experience"] = exp_objects
    return j_object["Clean Ids"], new_object

In [6]:
new_data = {}
verbose = Verbose(show_time=False, line_char="#", space_char=" ", arrow_head="")
length = len(json_data)
for i, officer in enumerate(json_data):
    verbose.make_verbose(i+1, length, start_with=officer["Clean Ids"])
    _id, obj = make_new_object(officer)
    new_data[_id] = obj


WB118X12  [#########################]  11569/11569

In [9]:
new_file = open("../datasets/formats/ias-json-new.json", "w", encoding="utf-8")

json.dump(new_data, new_file, indent=4)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../datasets/formats/ias-experience.csv")

In [3]:
p_start = []
p_end = []
for value in df["Period(From/To)"]:
    splited = value.split(" - ")
    p_start.append(splited[0])
    try:
        p_end.append(splited[1])
    except:
        p_end.append("N.A.")

In [5]:
df["Period_Start"] = p_start
df["Period_End"] = p_end

In [7]:
df = df.drop(columns=["Period(From/To)"])

In [9]:
df = df.rename({
    "Major Experience": "Major_Experience",
    "Minor Experience": "Minor_Experience"
})

In [14]:
df.to_csv("../datasets/formats/ias-experience.csv", index=False)